# 02 Correct Network Errors with Project Cards

In [1]:
import os
import sys
import yaml
import pickle
import glob

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

## Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'travel-model-two-networks')
root_dir = "Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'mtc-Lasso')
lasso_dir = "Z:/Data/Users/Sijia/MTC/github/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-12-08 18:02:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:02:02, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


## Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(input_dir, 'working_scenario_00.pickle')
v_00_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

In [7]:
v_00_scenario.road_net.links_df.shape

(1634769, 31)

In [8]:
v_00_scenario.transit_net.feed.routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 16 columns):
route_id_original         701 non-null object
agency_id                 526 non-null object
route_short_name          665 non-null object
route_long_name           679 non-null object
route_desc                14 non-null object
route_type                701 non-null int64
route_url                 217 non-null object
route_color               260 non-null object
route_text_color          243 non-null object
agency_raw_name           701 non-null object
route_sort_order          95 non-null object
min_headway_minutes       32 non-null object
eligibility_restricted    4 non-null object
continuous_pickup         4 non-null object
continuous_drop_off       4 non-null object
route_id                  701 non-null object
dtypes: int64(1), object(15)
memory usage: 87.8+ KB


## Create Scenario and Apply Project Cards

In [9]:
v_01_scenario = Scenario.create_scenario(
    base_scenario=v_00_scenario,
    card_directory = card_dir,
    validate_project_cards=False)

2020-12-08 18:03:06, INFO: Creating Scenario
2020-12-08 18:03:06, INFO: Creating Scenario


In [10]:
v_01_scenario.apply_all_projects()

2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:09, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2020-12-08 18:03:10, INFO: Applying Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma
2020-12-08 18:03:10, INFO: Applying Year 2015 Modify HOV L

2020-12-08 18:04:23, INFO: Applying Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-08 18:04:23, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-08 18:04:23, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on San Tomas Expressway NB Hwy 17 to I-880
2020-12-08 18:04:44, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-08 18:04:44, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-08 18:04:44, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-08 18:04:44, INFO: Applying Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-08 18:04:44, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Lawrence Expressway SB US 101 to I-280
2020-12-0

2020-12-08 18:05:39, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:39, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:39, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:39, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:39, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:39, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2020-12-08 18:05:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez Bridge Toll Plaza
2020-12-08 18:05:43, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez Bridge Toll Plaza
2020-12-08 18:05:43, INFO: Applying Year 2015 Modi

2020-12-08 18:06:17, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV lanes and USE on Hwy 4 WB from Deer Valley Road to Port Chicago Highway
2020-12-08 18:06:21, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18:06:21, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18:06:21, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18:06:21, INFO: Applying Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18:06:21, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18:06:21, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Hwy 4 EB from Port Chicago Highway to Deer Valley Road
2020-12-08 18

2020-12-08 18:07:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:22, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:22, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:22, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-237 WB North 1st Street to US 101
2020-12-08 18:07:24, INFO: Applying Year 2015 Modify Express Lanes and USE on CA-237 WB I-880 to CA-262 Mission Blvd
2020-12-08 18:07:24, INFO: Applying Year 2015 Modify Express Lanes and USE on CA-237 WB I-880 to CA-262 Mission Blvd
2020-12-08 18:07:24, INFO: Applying Year 201

2020-12-08 18:07:31, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-08 18:07:31, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-08 18:07:31, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-08 18:07:31, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2020-12-08 18:07:31, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-08 18:07:31, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-08 18:07:31, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-08 18:07:31, INFO: Applying Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-08 18:07:31, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Transbay Terminal Ramps
2020-12-08 18:07:31, INFO: Applying Project to Roadway Network: Year 2015 Bus Only S

In [11]:
v_01_scenario.applied_projects

['Year 2015 Modify HOV Lanes and USE on San Mateo Bridge Toll Plaza Toll Plaza on CA-92 WB - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass',
 'Year 2015 Modify HOV Lanes and USE on US-101 SB from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Oregon Expressway',
 'Year 2015 Modify HOV Lanes and USE on US 101 SB CA-85 to Cochrane Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB (Segment 02) from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Oregon Expressway to Whipple Road',
 'Year 2015 Modify HOV Lanes and USE on US-101 NB from North of Petaluma to South of Healdsburg',
 'Year 2015 Modify HOV Lanes and USE on US 101 NB Cochran Road to CA-85',
 'Year 2015 Modify HOV Lanes and USE on San Tomas Expressway SB I-8

In [12]:
v_01_scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'shape_id', 'geometry', 'ft_cal', 'ft',
       'useclass', 'assignable', 'transit', 'managed', 'ML_lanes',
       'segment_id', 'tollbooth', 'ML_useclass', 'ML_access', 'ML_egress',
       'tollseg', 'tollboth', 'bus_only', 'transit_access'],
      dtype='object')

## Make Travel Model Networks

In [13]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-12-08 18:13:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:13:14, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso


In [14]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(1634769, 42)
(644475, 11)
(869970, 7)


In [15]:
model_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-12-08 18:13:25, INFO: Adding centroid and centroid connector to standard network
2020-12-08 18:13:25, INFO: Adding centroid and centroid connector to standard network
2020-12-08 18:13:25, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:13:25, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:13:36, INFO: Finished adding centroid and centroid connectors
2020-12-08 18:13:36, INFO: Finished adding centroid and centroid connectors


In [16]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2190409, 42)
(688957, 11)
(1147790, 7)


In [17]:
model_net.links_df.model_link_id.nunique()

2190409

In [18]:
model_net = mtc.add_tap_and_tap_connector(
    roadway_network = model_net,
    parameters = parameters)

2020-12-08 18:13:47, INFO: Adding tap and tap connector to standard network
2020-12-08 18:13:47, INFO: Adding tap and tap connector to standard network
2020-12-08 18:13:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:13:47, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:13:55, INFO: Finished adding tap and tap connectors
2020-12-08 18:13:55, INFO: Finished adding tap and tap connectors


In [19]:
print(model_net.links_df.shape)
print(model_net.nodes_df.shape)
print(model_net.shapes_df.shape)

(2233923, 42)
(694957, 11)
(1169547, 7)


In [20]:
model_net.links_df.model_link_id.nunique()

2233923

In [21]:
model_net.nodes_df.model_node_id.nunique()

694957

In [22]:
model_net.shapes_df.id.nunique()

1169547

In [23]:
model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

2020-12-08 18:14:03, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-12-08 18:14:03, INFO: Renaming roadway attributes to be consistent with what mtc's model is expecting
2020-12-08 18:14:03, INFO: Creating managed lane network.
2020-12-08 18:14:03, INFO: Creating managed lane network.
2020-12-08 18:14:03, INFO: Creating network with duplicated managed lanes
2020-12-08 18:14:03, INFO: Creating network with duplicated managed lanes


c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\geopandas\geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
c:\users\wangs1\.conda\envs\lasso_mtc\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
c:\users\wang

2020-12-08 18:29:10, INFO: Determining cntype
2020-12-08 18:29:10, INFO: Determining cntype
2020-12-08 18:29:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:29:10, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:29:10, INFO: Calculating and adding roadway network variable: cntype
2020-12-08 18:29:10, INFO: Calculating and adding roadway network variable: cntype
2020-12-08 18:30:30, INFO: Finished determining variable: cntype
2020-12-08 18:30:30, INFO: Finished determining variable: cntype
2020-12-08 18:30:30, INFO: Determining transit
2020-12-08 18:30:30, INFO: Determining transit
2020-12-08 18:30:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:30:30, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-08 18:30:30, INFO: Variable 'transit' updated for some links. Returning without overwriting for those links. Calculating for other links
2

## Write to Disk

## As shapefile

In [48]:
model_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'links.shp'),
    output_node_shp = os.path.join(output_dir, 'nodes.shp'),
    link_output_variables = ["model_link_id", "A", "B", "geometry"],
    node_output_variables = ["model_node_id", "geometry"],
    data_to_csv = False,
    data_to_dbf = True,
)

2020-12-08 19:33:53, INFO: Writing Network as Shapefile
2020-12-08 19:33:53, INFO: Writing Network as Shapefile
2020-12-08 19:33:53, INFO: Renaming DBF Node Variables
2020-12-08 19:33:53, INFO: Renaming DBF Node Variables
2020-12-08 19:33:53, INFO: Renaming variables so that they are DBF-safe
2020-12-08 19:33:53, INFO: Renaming variables so that they are DBF-safe
2020-12-08 19:34:05, INFO: Renaming DBF Link Variables
2020-12-08 19:34:05, INFO: Renaming DBF Link Variables
2020-12-08 19:34:05, INFO: Renaming variables so that they are DBF-safe
2020-12-08 19:34:05, INFO: Renaming variables so that they are DBF-safe
2020-12-08 19:34:11, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_03\nodes.shp
2020-12-08 19:34:11, INFO: Writing Node Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks\data\processed\version_03\nodes.shp
2020-12-08 19:35:05, INFO: Writing Link Shapes:
 - Z:/Data/Users/Sijia/MTC/github/travel-mod

## As fixed width (for CUBE)

In [49]:
model_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'links.txt'),
    output_node_txt = os.path.join(output_dir, 'nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
    #drive_only = True
)

2020-12-08 19:40:00, INFO: Starting fixed width convertion
2020-12-08 19:40:00, INFO: Starting fixed width convertion
2020-12-08 20:02:53, INFO: Writing out link database
2020-12-08 20:02:53, INFO: Writing out link database
2020-12-08 20:03:29, INFO: Writing out link header and width ----
2020-12-08 20:03:29, INFO: Writing out link header and width ----
2020-12-08 20:03:29, INFO: Starting fixed width convertion
2020-12-08 20:03:29, INFO: Starting fixed width convertion
2020-12-08 20:05:14, INFO: Writing out node database
2020-12-08 20:05:14, INFO: Writing out node database
2020-12-08 20:05:21, INFO: Writing out node header and width
2020-12-08 20:05:21, INFO: Writing out node header and width


In [51]:
%%capture
os.chdir(output_dir)
!make_cube.bat

NODEMERGE: 4010555

NODEMERGE: 4011555

NODEMERGE: 4012555

NODEMERGE: 4013555

NODEMERGE: 4500722

NODEMERGE: 4501722

NODEMERGE: 4502722

NODEMERGE: 4503722

NODEMERGE: 4504722

NODEMERGE: 4505722

NODEMERGE: 4506722

NODEMERGE: 4507722

NODEMERGE: 4508722

In [52]:
%%capture
!runtpp make_simple_roadway_network.s
!runtpp make_complete_marin_network.s
!runtpp make_simple_transit_network.s



NODEMERGE: 4509722

NODEMERGE: 4510722

NODEMERGE: 4511722

NODEMERGE: 4512722

NODEMERGE: 4513722

NODEMERGE: 4514722

NODEMERGE: 4515722

NODEMERGE: 4516722

NODEMERGE: 4517722

NODEMERGE: 4518722

NODEMERGE: 4519722

NODEMERGE: 4520722

NODEMERGE: 4521722

NODEMERGE: 4522722

NODEMERGE: 4523722

NODEMERGE: 4524722

NODEMERGE: 4525722

NODEMERGE: 4526722

NODEMERGE: 4527722

NODEMERGE: 4528722

NODEMERGE: 4529722

NODEMERGE: 4530722

NODEMERGE: 4531722

NODEMERGE: 4532722

NODEMERGE: 4533722

NODEMERGE: 4534722

NODEMERGE: 4535722

NODEMERGE: 4536722

NODEMERGE: 4537722

NODEMERGE: 4538722

NODEMERGE: 4539722

NODEMERGE: 4540722

NODEMERGE: 4541722

NODEMERGE: 4542722

NODEMERGE: 4543722

NODEMERGE: 4544722

NODEMERGE: 4545722

NODEMERGE: 4546722

NODEMERGE: 4547722

NODEMERGE: 4548722

NODEMERGE: 4549722

NODEMERGE: 4550722

NODEMERGE: 4551722

NODEMERGE: 4552722

NODEMERGE: 4553722

NODEMERGE: 4554722

NODEMERGE: 4555722

NODEMERGE: 5000576

NODEMERGE: 5001576

NODEMERGE: 5002576


LINKMERGE: 2000154-2085605     

LINKMERGE: 2000452-2012663     

LINKMERGE: 2000741-2117244     

LINKMERGE: 2001032-2048852     

LINKMERGE: 2001328-2184955     

LINKMERGE: 2001627-2026574     

LINKMERGE: 2001919-2035938     

LINKMERGE: 2002218-2039067     

LINKMERGE: 2002519-2135699     

LINKMERGE: 2002824-2171514     

LINKMERGE: 2003127-2040544     

LINKMERGE: 2003421-2171993     

LINKMERGE: 2003718-2089243     

LINKMERGE: 2004008-2130202     

LINKMERGE: 2004304-2120913     

LINKMERGE: 2004609-2095056     

LINKMERGE: 2004902-2010519     

LINKMERGE: 2005196-2139581     

LINKMERGE: 2005487-2140411     

LINKMERGE: 2005782-2067929     

LINKMERGE: 2006076-2127488     

LINKMERGE: 2006372-2035223     

LINKMERGE: 2006665-2066454     

LINKMERGE: 2006963-2106763     

LINKMERGE: 2007255-2149947     

LINKMERGE: 2007549-2151860     

LINKMERGE: 2007843-2152975     

LINKMERGE: 2008146-2180251     

LINKMERGE: 2008441-2101370     

LINKMERGE: 2008736-2048705     

LINKMERGE


LINKMERGE: 2160507-2168365     

LINKMERGE: 2160840-2033930     

LINKMERGE: 2161170-2103227     

LINKMERGE: 2161508-2025836     

LINKMERGE: 2161838-2094969     

LINKMERGE: 2162181-2148158     

LINKMERGE: 2162529-2020920     

LINKMERGE: 2162869-2027410     

LINKMERGE: 2163214-2114360     

LINKMERGE: 2163555-2084842     

LINKMERGE: 2163899-2118155     

LINKMERGE: 2164243-229477     

LINKMERGE: 2164583-2091291     

LINKMERGE: 2164932-2023905     

LINKMERGE: 2165279-2040572     

LINKMERGE: 2165627-2009737     

LINKMERGE: 2165973-2013443     

LINKMERGE: 2166331-290578     

LINKMERGE: 2166675-2082743     

LINKMERGE: 2167022-2052414     

LINKMERGE: 2167361-2005935     

LINKMERGE: 2167734-2017509     

LINKMERGE: 2168086-2038008     

LINKMERGE: 2168443-2038388     

LINKMERGE: 2168796-2044839     

LINKMERGE: 2169163-2102116     

LINKMERGE: 2169519-2118264     

LINKMERGE: 2169861-2037143     

LINKMERGE: 2170217-2045090     

LINKMERGE: 2170569-2000580     

LINKMERGE: 

## As transit line file (for CUBE)

In [50]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
mtc.write_as_cube_lin(standard_transit_net, parameters, outpath = os.path.join(output_dir, "transit.lin"))

2020-12-09 09:21:49, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-09 09:21:49, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/MTC/github/Lasso
2020-12-09 09:21:49, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
2020-12-09 09:21:49, INFO: Converting GTFS Standard Properties to MTC's Cube Standard
Empty DataFrame
Columns: [service_id, trip_headsign, direction_id, block_id, wheelchair_accessible, bikes_allowed, agency_raw_name, trip_short_name, original_trip_id, trip_bikes_allowed, trip_type, drt_max_travel_time, drt_avg_travel_time, drt_advance_book_min, drt_pickup_message, drt_drop_off_message, continuous_pickup_message, continuous_drop_off_message, route_id, trip_id, shape_id, route_id_original, agency_id, route_short_name, route_long_name, route_desc, route_type, route_url, route_color, route_text_color, route_sort_order, min_headway_minutes, eligibility_restricted, continuous_pickup, continuous_drop_off, headway_secs, start

## As pickle

In [53]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(working_scenario_filename, 'wb'))